In [2]:
%load_ext kedro.extras.extensions.ipython

2022-08-25 12:14:46,211 - kedro.framework.hooks.manager - INFO - Registered hooks from 1 installed plugin(s): kedro-mlflow-0.8.1
2022-08-25 12:14:46,365 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.


/home/michal/projects/gid-ml-framework/.venv/lib/python3.8/site-packages/kedro/framework/context/context.py:488: UserWarning: Credentials not found in your Kedro project config.
No files found in ['/home/michal/projects/gid-ml-framework/conf/base', '/home/michal/projects/gid-ml-framework/conf/local'] matching the glob pattern(s): ['credentials*', 'credentials*/**', '**/credentials*']
  warn(f"Credentials not found in your Kedro project config.\n{str(exc)}")


2022-08-25 12:14:46,624 - root - INFO - ** Kedro project GetInData ML Framework
2022-08-25 12:14:46,626 - root - INFO - Defined global variable `context`, `session`, `catalog` and `pipelines`
2022-08-25 12:14:48,489 - root - INFO - Registered line magic `run_viz`
2022-08-25 12:14:48,491 - root - INFO - Registered line magic `reload_kedro_mlflow`


/home/michal/projects/gid-ml-framework/.venv/lib/python3.8/site-packages/pkg_resources/_vendor/packaging/specifiers.py:255: DeprecationWarning: Creating a LegacyVersion has been deprecated and will be removed in the next major release
  warnings.warn(


In [3]:
%reload_kedro

2022-08-25 12:14:48,824 - kedro.framework.hooks.manager - INFO - Registered hooks from 1 installed plugin(s): kedro-mlflow-0.8.1
2022-08-25 12:14:48,948 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.
2022-08-25 12:14:49,007 - root - INFO - ** Kedro project GetInData ML Framework
2022-08-25 12:14:49,009 - root - INFO - Defined global variable `context`, `session`, `catalog` and `pipelines`
2022-08-25 12:14:49,033 - root - INFO - Registered line magic `run_viz`
2022-08-25 12:14:49,036 - root - INFO - Registered line magic `reload_kedro_mlflow`


/home/michal/projects/gid-ml-framework/.venv/lib/python3.8/site-packages/kedro/framework/context/context.py:488: UserWarning: Credentials not found in your Kedro project config.
No files found in ['/home/michal/projects/gid-ml-framework/conf/base', '/home/michal/projects/gid-ml-framework/conf/local'] matching the glob pattern(s): ['credentials*', 'credentials*/**', '**/credentials*']
  warn(f"Credentials not found in your Kedro project config.\n{str(exc)}")


In [4]:
from typing import Iterator, Union, Tuple, List
from datetime import datetime

import pandas as pd
import numpy as np
from kedro.extras.datasets.pandas import CSVDataSet
from kedro.io.core import get_filepath_str
from sklearn.model_selection import train_test_split

import gid_ml_framework.pipelines.santander_preprocessing.nodes
from gid_ml_framework.extras.datasets.chunks_dataset import (
 _load,
 _concat_chunks,
)

pd.options.mode.chained_assignment = None
pd.set_option('expand_frame_repr', True)
pd.set_option("display.max_rows", 999)
pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.expand_frame_repr', False)

## Checking input dtypes

In [52]:
input_train_path = "santander_train_input"

In [53]:
train_input = _concat_chunks(context.catalog.load(input_train_path))

2022-08-25 11:34:02,664 - kedro.io.data_catalog - INFO - Loading data from `santander_train_input` (CSVDataSet)...


/home/michal/projects/gid-ml-framework/.venv/lib/python3.8/site-packages/kedro/framework/context/context.py:488: UserWarning: Credentials not found in your Kedro project config.
No files found in ['/home/michal/projects/gid-ml-framework/conf/base', '/home/michal/projects/gid-ml-framework/conf/local'] matching the glob pattern(s): ['credentials*', 'credentials*/**', '**/credentials*']
  warn(f"Credentials not found in your Kedro project config.\n{str(exc)}")


In [57]:
train_input.head()

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,ult_fec_cli_1t,indrel_1mes,tiprel_1mes,indresi,indext,conyuemp,canal_entrada,indfall,tipodom,cod_prov,nomprov,ind_actividad_cliente,renta,segmento,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,2015-01-28,1375586,N,ES,H,35,2015-01-12,0,6,1,NaN,1.0,A,S,N,NaN,KHL,N,1,29.0,MALAGA,1,87218.101562,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2015-01-28,1050611,N,ES,V,23,2012-08-10,0,35,1,NaN,1.0,I,S,S,NaN,KHE,N,1,13.0,CIUDAD REAL,0,35548.738281,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2015-01-28,1050612,N,ES,V,23,2012-08-10,0,35,1,NaN,1.0,I,S,N,NaN,KHE,N,1,13.0,CIUDAD REAL,0,122179.109375,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2015-01-28,1050613,N,ES,H,22,2012-08-10,0,35,1,NaN,1.0,I,S,N,NaN,KHD,N,1,50.0,ZARAGOZA,0,119775.539062,03 - UNIVERSITARIO,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2015-01-28,1050614,N,ES,V,23,2012-08-10,0,35,1,NaN,1.0,A,S,N,NaN,KHE,N,1,50.0,ZARAGOZA,1,NaN,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [18]:
train_input.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13647309 entries, 0 to 13647308
Data columns (total 48 columns):
 #   Column                 Dtype   
---  ------                 -----   
 0   fecha_dato             object  
 1   ncodpers               int32   
 2   ind_empleado           object  
 3   pais_residencia        object  
 4   sexo                   category
 5   age                    object  
 6   fecha_alta             object  
 7   ind_nuevo              category
 8   antiguedad             object  
 9   indrel                 UInt8   
 10  ult_fec_cli_1t         object  
 11  indrel_1mes            object  
 12  tiprel_1mes            object  
 13  indresi                object  
 14  indext                 category
 15  conyuemp               object  
 16  canal_entrada          object  
 17  indfall                object  
 18  tipodom                UInt8   
 19  cod_prov               float64 
 20  nomprov                object  
 21  ind_actividad_cliente  UInt8 

In [19]:
train_input.memory_usage(deep=True)

Index                          128
fecha_dato               914369703
ncodpers                  54589236
ind_empleado             790822838
pais_residencia          804442413
sexo                      13647425
age                      587303124
fecha_alta               913399013
ind_nuevo                 13647427
antiguedad               602766580
indrel                    27294618
ult_fec_cli_1t           437581643
indrel_1mes              724455212
tiprel_1mes              787649616
indresi                  790822838
indext                    13647425
conyuemp                 436760896
canal_entrada            813627012
indfall                  790822838
tipodom                   27294618
cod_prov                 109178472
nomprov                  884970952
ind_actividad_cliente     27294618
renta                     54589236
segmento                  13647631
ind_ahor_fin_ult1         27294618
ind_aval_fin_ult1         27294618
ind_cco_fin_ult1          27294618
ind_cder_fin_ult1   

: 

In [ ]:
train_input.isnull()

## Comparing cleaned train and test dataframes

In [5]:
cleaned_train_path = "santander_preprocessing_train.santander_cleaned"
cleaned_test_path = "santander_preprocessing_test.santander_cleaned"

In [6]:
cleaned_train = _concat_chunks(context.catalog.load(cleaned_train_path))
cleaned_test = _concat_chunks(context.catalog.load(cleaned_test_path))

2022-08-25 12:17:10,085 - kedro.io.data_catalog - INFO - Loading data from `santander_preprocessing_train.santander_cleaned` (CSVDataSet)...


/home/michal/projects/gid-ml-framework/.venv/lib/python3.8/site-packages/kedro/framework/context/context.py:488: UserWarning: Credentials not found in your Kedro project config.
No files found in ['/home/michal/projects/gid-ml-framework/conf/base', '/home/michal/projects/gid-ml-framework/conf/local'] matching the glob pattern(s): ['credentials*', 'credentials*/**', '**/credentials*']
  warn(f"Credentials not found in your Kedro project config.\n{str(exc)}")


2022-08-25 12:17:29,878 - kedro.io.data_catalog - INFO - Loading data from `santander_preprocessing_test.santander_cleaned` (CSVDataSet)...


/home/michal/projects/gid-ml-framework/.venv/lib/python3.8/site-packages/kedro/framework/context/context.py:488: UserWarning: Credentials not found in your Kedro project config.
No files found in ['/home/michal/projects/gid-ml-framework/conf/base', '/home/michal/projects/gid-ml-framework/conf/local'] matching the glob pattern(s): ['credentials*', 'credentials*/**', '**/credentials*']
  warn(f"Credentials not found in your Kedro project config.\n{str(exc)}")


In [8]:
cleaned_train.isnull().sum()

fecha_dato                     0
ncodpers                       0
ind_empleado                 103
pais_residencia              103
sexo                         103
age                            0
fecha_alta                   103
ind_nuevo                    103
antiguedad                     0
indrel                       103
ult_fec_cli_1t           1352020
indrel_1mes                11373
tiprel_1mes                11373
indresi                      103
indext                       103
conyuemp                 1352034
canal_entrada              14707
indfall                      103
nomprov                     6729
ind_actividad_cliente        103
renta                     274547
segmento                   14986
ind_ahor_fin_ult1              0
ind_aval_fin_ult1              0
ind_cco_fin_ult1               0
ind_cder_fin_ult1              0
ind_cno_fin_ult1               0
ind_ctju_fin_ult1              0
ind_ctma_fin_ult1              0
ind_ctop_fin_ult1              0
ind_ctpp_f

In [32]:
np.unique(cleaned_train.loc[:, "renta"])

array([3.73179000e+03, 3.79686000e+03, 6.61158000e+03, ...,
       2.20347388e+07, 2.55472519e+07,            nan])

In [27]:
np.unique(cleaned_test.loc[:, "renta"])

array(['         NA', '    1202.73', '    1471.23', ..., '24253236.93',
       '25547251.92', '28894395.51'], dtype=object)

In [34]:
df = cleaned_train.copy()
mask = df['renta'].map(lambda x: isinstance(x, (int, float)))
df.loc[:, 'renta'] = df.loc[:, 'renta'].where(mask)
np.unique(df.loc[:, "renta"])

array([3.73179000e+03, 3.79686000e+03, 6.61158000e+03, ...,
       2.20347388e+07, 2.55472519e+07,            nan])

In [42]:
df = cleaned_test
pd.to_numeric(df.loc[:, 'renta'], downcast="float", errors='coerce')

0         326124.906250
1                   NaN
2                   NaN
3         148402.984375
4         106885.796875
              ...      
929610    128643.570312
929611              NaN
929612     72765.273438
929613    147488.875000
929614              NaN
Name: renta, Length: 929615, dtype: float32

## Comparing final dataframes

In [10]:
train_path = "santander_train"
val_path = "santander_val"
test_path = "santander_test"

In [11]:
train = _concat_chunks(context.catalog.load(train_path))
val = _concat_chunks(context.catalog.load(val_path))
test = _concat_chunks(context.catalog.load(test_path))

2022-08-25 12:29:35,998 - kedro.io.data_catalog - INFO - Loading data from `santander_train` (CSVDataSet)...


/home/michal/projects/gid-ml-framework/.venv/lib/python3.8/site-packages/kedro/framework/context/context.py:488: UserWarning: Credentials not found in your Kedro project config.
No files found in ['/home/michal/projects/gid-ml-framework/conf/base', '/home/michal/projects/gid-ml-framework/conf/local'] matching the glob pattern(s): ['credentials*', 'credentials*/**', '**/credentials*']
  warn(f"Credentials not found in your Kedro project config.\n{str(exc)}")


2022-08-25 12:29:55,050 - kedro.io.data_catalog - INFO - Loading data from `santander_val` (CSVDataSet)...


/home/michal/projects/gid-ml-framework/.venv/lib/python3.8/site-packages/kedro/framework/context/context.py:488: UserWarning: Credentials not found in your Kedro project config.
No files found in ['/home/michal/projects/gid-ml-framework/conf/base', '/home/michal/projects/gid-ml-framework/conf/local'] matching the glob pattern(s): ['credentials*', 'credentials*/**', '**/credentials*']
  warn(f"Credentials not found in your Kedro project config.\n{str(exc)}")


2022-08-25 12:29:56,579 - kedro.io.data_catalog - INFO - Loading data from `santander_test` (CSVDataSet)...


/home/michal/projects/gid-ml-framework/.venv/lib/python3.8/site-packages/kedro/framework/context/context.py:488: UserWarning: Credentials not found in your Kedro project config.
No files found in ['/home/michal/projects/gid-ml-framework/conf/base', '/home/michal/projects/gid-ml-framework/conf/local'] matching the glob pattern(s): ['credentials*', 'credentials*/**', '**/credentials*']
  warn(f"Credentials not found in your Kedro project config.\n{str(exc)}")


In [30]:
train.head()

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,ult_fec_cli_1t,indrel_1mes,tiprel_1mes,indresi,indext,conyuemp,canal_entrada,indfall,nomprov,ind_actividad_cliente,renta,segmento,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,2015-01-28,1050610,N,ES,V,24,2012-08-10,0,35,1,UNKNOWN,1,I,S,N,UNKNOWN,KHE,N,SALAMANCA,1,68318.460938,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2015-01-28,17045,N,ES,H,54,1995-05-09,0,242,1,UNKNOWN,1,A,S,N,UNKNOWN,KAT,N,MADRID,1,403907.906250,01 - TOP,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,2015-01-28,16019,A,ES,V,56,1995-01-31,0,246,1,UNKNOWN,1,A,S,N,UNKNOWN,KAT,N,MADRID,1,256018.078125,01 - TOP,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,1,1,0,1,1,1
3,2015-01-28,16038,N,ES,V,50,1995-02-20,0,245,1,UNKNOWN,1,A,S,N,UNKNOWN,KAT,N,MADRID,0,223767.156250,02 - PARTICULARES,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2015-01-28,16039,N,ES,H,56,1995-02-20,0,245,1,UNKNOWN,1,A,S,N,UNKNOWN,KAT,N,MADRID,1,196300.343750,01 - TOP,0,0,0,0,1,0,0,0,1,0,1,1,0,1,1,1,0,0,1,0,0,1,1,1


In [32]:
train.dtypes

fecha_dato                 object
ncodpers                    int32
ind_empleado               object
pais_residencia            object
sexo                       object
age                         int64
fecha_alta                 object
ind_nuevo                category
antiguedad                  int64
indrel                      UInt8
ult_fec_cli_1t             object
indrel_1mes                object
tiprel_1mes                object
indresi                    object
indext                     object
conyuemp                   object
canal_entrada              object
indfall                    object
nomprov                    object
ind_actividad_cliente       UInt8
renta                     float32
segmento                   object
ind_ahor_fin_ult1           UInt8
ind_aval_fin_ult1           UInt8
ind_cco_fin_ult1            UInt8
ind_cder_fin_ult1           UInt8
ind_cno_fin_ult1            UInt8
ind_ctju_fin_ult1           UInt8
ind_ctma_fin_ult1           UInt8
ind_ctop_fin_u

In [38]:
val.head()

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,ult_fec_cli_1t,indrel_1mes,tiprel_1mes,indresi,indext,conyuemp,canal_entrada,indfall,nomprov,ind_actividad_cliente,renta,segmento,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1,month
0,2016-05-28,657794,N,ES,H,49,2006-11-18,0,114,1,UNKNOWN,1,A,S,N,UNKNOWN,KAT,N,BARCELONA,1,102189.000000,01 - TOP,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,5
1,2016-05-28,1549195,N,ES,V,32,2016-05-05,1,0,1,UNKNOWN,P,A,S,N,UNKNOWN,UNKNOWN,N,VALLADOLID,0,92352.570312,UNKNOWN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5
2,2016-05-28,1549547,N,ES,H,66,2016-05-08,1,0,1,UNKNOWN,P,A,S,N,UNKNOWN,UNKNOWN,N,PONTEVEDRA,0,98407.843750,UNKNOWN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5
3,2016-05-28,1548962,N,ES,H,76,2016-05-04,1,0,1,UNKNOWN,P,A,S,N,UNKNOWN,UNKNOWN,N,MADRID,1,138007.234375,UNKNOWN,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5
4,2016-05-28,1548965,N,ES,H,57,2016-05-04,1,0,1,UNKNOWN,P,A,S,N,UNKNOWN,UNKNOWN,N,MADRID,1,138007.234375,UNKNOWN,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5


In [34]:
val.dtypes

fecha_dato                 object
ncodpers                    int32
ind_empleado               object
pais_residencia            object
sexo                       object
age                         int64
fecha_alta                 object
ind_nuevo                category
antiguedad                  int64
indrel                      UInt8
ult_fec_cli_1t             object
indrel_1mes                object
tiprel_1mes                object
indresi                    object
indext                     object
conyuemp                   object
canal_entrada              object
indfall                    object
nomprov                    object
ind_actividad_cliente       UInt8
renta                     float32
segmento                 category
ind_ahor_fin_ult1           UInt8
ind_aval_fin_ult1           UInt8
ind_cco_fin_ult1            UInt8
ind_cder_fin_ult1           UInt8
ind_cno_fin_ult1            UInt8
ind_ctju_fin_ult1           UInt8
ind_ctma_fin_ult1           UInt8
ind_ctop_fin_u

In [37]:
test.head()

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,ult_fec_cli_1t,indrel_1mes,tiprel_1mes,indresi,indext,conyuemp,canal_entrada,indfall,nomprov,ind_actividad_cliente,renta,segmento,month
0,2016-06-28,15889,F,ES,V,56,1995-01-16,0,256,1,UNKNOWN,1,A,S,N,N,KAT,N,MADRID,1,326124.906250,01 - TOP,6
1,2016-06-28,28067,N,ES,V,43,1995-11-27,0,247,1,UNKNOWN,1,A,S,N,UNKNOWN,KAT,N,BARCELONA,1,50724.359375,01 - TOP,6
2,2016-06-28,28068,N,ES,H,60,1995-11-27,0,247,1,UNKNOWN,1,A,S,N,UNKNOWN,KAT,N,MADRID,1,75976.828125,02 - PARTICULARES,6
3,2016-06-28,28070,N,ES,V,45,1995-11-27,0,247,1,UNKNOWN,1,I,S,N,UNKNOWN,KAT,N,VALENCIA,0,62352.421875,02 - PARTICULARES,6
4,2016-06-28,28077,N,ES,H,56,1995-11-29,0,247,1,UNKNOWN,1,A,S,N,UNKNOWN,KAT,N,"CORUÑA, A",1,80161.617188,01 - TOP,6


In [33]:
test.dtypes

fecha_dato                 object
ncodpers                    int32
ind_empleado               object
pais_residencia            object
sexo                       object
age                         int64
fecha_alta                 object
ind_nuevo                category
antiguedad                  int64
indrel                      UInt8
ult_fec_cli_1t             object
indrel_1mes                object
tiprel_1mes                object
indresi                    object
indext                     object
conyuemp                   object
canal_entrada              object
indfall                    object
nomprov                    object
ind_actividad_cliente       UInt8
renta                     float64
segmento                 category
dtype: object

In [12]:
test.dtypes == train.loc[:, test.columns].dtypes

fecha_dato               True
ncodpers                 True
ind_empleado             True
pais_residencia          True
sexo                     True
age                      True
fecha_alta               True
ind_nuevo                True
antiguedad               True
indrel                   True
ult_fec_cli_1t           True
indrel_1mes              True
tiprel_1mes              True
indresi                  True
indext                   True
conyuemp                 True
canal_entrada            True
indfall                  True
nomprov                  True
ind_actividad_cliente    True
renta                    True
segmento                 True
dtype: bool

In [13]:
test.dtypes == val.loc[:, test.columns].dtypes

fecha_dato               True
ncodpers                 True
ind_empleado             True
pais_residencia          True
sexo                     True
age                      True
fecha_alta               True
ind_nuevo                True
antiguedad               True
indrel                   True
ult_fec_cli_1t           True
indrel_1mes              True
tiprel_1mes              True
indresi                  True
indext                   True
conyuemp                 True
canal_entrada            True
indfall                  True
nomprov                  True
ind_actividad_cliente    True
renta                    True
segmento                 True
dtype: bool